In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten,BatchNormalization
from keras.callbacks import EarlyStopping

from keras.constraints import maxnorm
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
seed = 16
filename = '../Datos/fer2013.csv'
label_map = ['Enfado', 'Asco', 'Miedo', 'Alegria', 'Tristeza', 'Sorpresa', 'Neutral']
names=['emocion','pixels','uso']
df=pd.read_csv(filename,names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emocion,pixels,uso
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [3]:
def getData(filname):
    # Las imagenes 48x48 en grayscale (de 0 a 255)
    # N = 35887 (28709 de entrenamiento y 2304 de testeo)
    Y_train = []
    X_train = []
    Y_test = []
    X_test = []
    
    first = True
    uso = 'Training\n'
    for line in open(filname):
        #This condition skips the first condition
        if first:
            first = False
        else:
            row = line.split(',')
            condition = str(row[2])
            if condition == 'Training\n':
                Y_train.append(int(row[0]))
                X_train.append([int(p) for p in row[1].split()])
            else:
                Y_test.append(int(row[0]))
                X_test.append([int(p) for p in row[1].split()])
                
    X_train, Y_train = np.array(X_train)/255.0 , np.array(Y_train)
    X_test, Y_test = np.array(X_test)/255.0 , np.array(Y_test)

    return X_train, X_test, Y_train, Y_test 

In [4]:
X_train, X_test, Y_train, Y_test= getData(filename)

In [5]:
N_train = X_train.shape[0]
N_test = X_test.shape[0]
X_train = X_train.reshape(N_train, 48, 48,1)
X_test = X_test.reshape(N_test, 48, 48,1)

In [6]:
cont_train = [0,0,0,0,0,0,0]
for j in Y_train:
    cont_train[j] += 1
cont_test = [0,0,0,0,0,0,0]
for j in Y_test:
    cont_test[j] += 1

print(cont_train)
print(cont_test)

[3995, 436, 4097, 7215, 4830, 3171, 4965]
[958, 111, 1024, 1774, 1247, 831, 1233]


In [7]:
# one hot encode output
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)
class_num = Y_test.shape[1]

In [8]:
model = Sequential()

model.add(Conv2D(6,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'tanh'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(16,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'tanh'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='tanh',kernel_constraint=maxnorm(3)))
model.add(Dense(class_num, activation='softmax',kernel_constraint=maxnorm(3)))

In [9]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 44, 44, 6)         156       
_________________________________________________________________
average_pooling2d (AveragePo (None, 22, 22, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 18, 16)        2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 9, 9, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1296)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               166016    
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 9

In [10]:
epochs = 100
optimizer = 'adam'
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.1, verbose = 1, patience = 10)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [11]:
np.random.seed(16)
cb =  [es]
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64, callbacks = cb)

Epoch 1/100
449/449 [==============================] - 29s 65ms/step - loss: 1.7022 - accuracy: 0.3292 - val_loss: 1.6436 - val_accuracy: 0.3671
Epoch 2/100
449/449 [==============================] - 30s 68ms/step - loss: 1.6379 - accuracy: 0.3657 - val_loss: 1.6280 - val_accuracy: 0.3792
Epoch 3/100
449/449 [==============================] - 29s 64ms/step - loss: 1.6247 - accuracy: 0.3708 - val_loss: 1.6185 - val_accuracy: 0.3777
Epoch 4/100
449/449 [==============================] - 29s 64ms/step - loss: 1.6168 - accuracy: 0.3743 - val_loss: 1.6179 - val_accuracy: 0.3807
Epoch 5/100
449/449 [==============================] - 29s 64ms/step - loss: 1.6130 - accuracy: 0.3766 - val_loss: 1.6197 - val_accuracy: 0.3746
Epoch 6/100
449/449 [==============================] - 29s 64ms/step - loss: 1.6060 - accuracy: 0.3801 - val_loss: 1.6222 - val_accuracy: 0.3721
Epoch 7/100
449/449 [==============================] - 29s 64ms/step - loss: 1.6012 - accuracy: 0.3826 - val_loss: 1.6178 - val_ac

In [12]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 40.16%


In [13]:
model = Sequential()

model.add(Conv2D(6,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(16,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dense(class_num, activation='softmax',kernel_constraint=maxnorm(3)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 44, 44, 6)         156       
_________________________________________________________________
average_pooling2d_2 (Average (None, 22, 22, 6)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 16)        2416      
_________________________________________________________________
average_pooling2d_3 (Average (None, 9, 9, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1296)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               166016    
_________________________________________________________________
dense_3 (Dense)              (None, 7)                

In [14]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64, callbacks = cb)

Epoch 1/100
449/449 [==============================] - 28s 62ms/step - loss: 1.6903 - accuracy: 0.3334 - val_loss: 1.6023 - val_accuracy: 0.3816
Epoch 2/100
449/449 [==============================] - 30s 67ms/step - loss: 1.5501 - accuracy: 0.4009 - val_loss: 1.5149 - val_accuracy: 0.4136
Epoch 3/100
449/449 [==============================] - 30s 66ms/step - loss: 1.4799 - accuracy: 0.4294 - val_loss: 1.4803 - val_accuracy: 0.4259
Epoch 4/100
449/449 [==============================] - 30s 68ms/step - loss: 1.4241 - accuracy: 0.4521 - val_loss: 1.4354 - val_accuracy: 0.4533
Epoch 5/100
449/449 [==============================] - 29s 65ms/step - loss: 1.3699 - accuracy: 0.4766 - val_loss: 1.4056 - val_accuracy: 0.4611
Epoch 6/100
449/449 [==============================] - 28s 63ms/step - loss: 1.3238 - accuracy: 0.4933 - val_loss: 1.3823 - val_accuracy: 0.4677
Epoch 7/100
449/449 [==============================] - 28s 62ms/step - loss: 1.2758 - accuracy: 0.5146 - val_loss: 1.3684 - val_ac

In [15]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 48.55%


In [16]:
model = Sequential()

model.add(Conv2D(6,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dense(class_num, activation='softmax',kernel_constraint=maxnorm(3)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 44, 44, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 6)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 18, 18, 16)        2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1296)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               166016    
_________________________________________________________________
dense_5 (Dense)              (None, 7)                

In [17]:

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64, callbacks = cb)

Epoch 1/100
449/449 [==============================] - 28s 61ms/step - loss: 1.6452 - accuracy: 0.3574 - val_loss: 1.5195 - val_accuracy: 0.4142
Epoch 2/100
449/449 [==============================] - 31s 68ms/step - loss: 1.4764 - accuracy: 0.4335 - val_loss: 1.4249 - val_accuracy: 0.4537
Epoch 3/100
449/449 [==============================] - 26s 58ms/step - loss: 1.3935 - accuracy: 0.4669 - val_loss: 1.3796 - val_accuracy: 0.4703
Epoch 4/100
449/449 [==============================] - 29s 65ms/step - loss: 1.3355 - accuracy: 0.4897 - val_loss: 1.3650 - val_accuracy: 0.4734
Epoch 5/100
449/449 [==============================] - 28s 62ms/step - loss: 1.2810 - accuracy: 0.5121 - val_loss: 1.3514 - val_accuracy: 0.4844
Epoch 6/100
449/449 [==============================] - 29s 64ms/step - loss: 1.2351 - accuracy: 0.5328 - val_loss: 1.3092 - val_accuracy: 0.4980
Epoch 7/100
449/449 [==============================] - 29s 64ms/step - loss: 1.1888 - accuracy: 0.5516 - val_loss: 1.3116 - val_ac

In [18]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 50.45%
